In [25]:
import pandas as pd 

In [26]:
tubercolusis = pd.read_excel('..\data\Incidencia_de_tuberculosis_por_100_000_hab.xls')

In [27]:
tubercolusis

,Incidencia de tuberculosis por 100 000 hab.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,CC.AA.,Años,NaN,NaN,NaN,NaN
1,NaN,2017,2018,2019,2020,2021
2,NaN,Total,Total,Total,Total,Total
3,España (ES),9.42,9.39,9.28,7.58,7.61
4,Andalucía (AN),6.59,7.19,6.46,5.93,5.53
5,Aragón (AR),10.18,8.59,10.12,6.31,8.45
6,"Asturias, Principado de (AS)",8.35,10.15,10.49,11.72,7.73
7,"Balears, Illes (IB)",8.98,9.69,5.42,6.09,5.82
8,Canarias (CN),5.13,7.04,5.72,4.54,3.25
9,Cantabria (CB),9.81,13.08,8.08,6.53,5.99


In [28]:
# FUNCION DE LIMPIEZA GENERAL:

def limpieza_columnas(df):
    
    df_copy = df.copy()
    
    years = df_copy.iloc[1].values
    genders = df_copy.iloc[2].values
    
    new_columns = []
    current_year = None
    
    # SEPARACION AÑOS Y GÉNERO
    
    for i in range(len(years)):
        if pd.isna(years[i]):
            new_columns.append(f'{current_year}_{genders[i]}')
        else:
            current_year = years[i]
            new_columns.append(f'{current_year}_{genders[i]}')
    
    df_copy.columns = new_columns
    
    df_copy = df_copy.iloc[3:]
    
    df_copy.set_index(df_copy.columns[0], inplace=True)
    
    # Eliminar las tres últimas filas si tienen más de un 90% de valores nulos, hay algunos excel que tienen nulos al final como observaciones
    
    rows_to_drop = df_copy.tail(3).index[df_copy.tail(3).isnull().mean(axis=1) > 0.9]
    df_copy = df_copy.drop(rows_to_drop)
    
    # QUITAR MULTIINDEX
    
    df_copy = df_copy.reset_index()
      
    df_copy.rename(columns={'None_nan': 'Region'}, inplace=True)
      
    return df_copy

test = limpieza_columnas(tubercolusis)

test.head()

,Region,2017_Total,2018_Total,2019_Total,2020_Total,2021_Total
0,España (ES),9.42,9.39,9.28,7.58,7.61
1,Andalucía (AN),6.59,7.19,6.46,5.93,5.53
2,Aragón (AR),10.18,8.59,10.12,6.31,8.45
3,"Asturias, Principado de (AS)",8.35,10.15,10.49,11.72,7.73
4,"Balears, Illes (IB)",8.98,9.69,5.42,6.09,5.82


In [30]:
def nuevos_df(df):
    
    # Verifica si alguna de las columnas contiene la palabra 'Hombre' para localizar los df diferentes:
    
    if all('Hombre' not in col for col in df.columns):
        
        df_new = pd.DataFrame(columns=['Region', 'Año', 'Total'])
        
        years = set(col.split('_')[0] for col in df.columns if '_' in col)  # cogemos año 
        
        for year in years:
            
            if f'{year}_Total' in df.columns:
                
                df_year = df[['Region', f'{year}_Total']].copy()
                                
                df_year.columns = ['Region', 'Total']
                df_year['Año'] = year
                df_new = pd.concat([df_new, df_year], ignore_index=True)
                
    else:
        
        df_new = pd.DataFrame(columns=['Region', 'Año', 'Hombre', 'Mujer', 'Total'])
       
        years = set(col.split('_')[0] for col in df.columns if '_' in col)  # cogemos año 
    
        for year in years:
        
            if f'{year}_Hombres' in df.columns and f'{year}_Mujeres' in df.columns and f'{year}_Total' in df.columns:
            
                df_year = df[['Region', f'{year}_Hombres', f'{year}_Mujeres', f'{year}_Total']].copy()
                df_year.columns = ['Region', 'Hombre', 'Mujer', 'Total']
                df_year['Año'] = year
                df_new = pd.concat([df_new, df_year], ignore_index=True)
                
    return df_new

prueba = nuevos_df(test)

prueba.head()

,Region,Año,Total
0,España (ES),2021,7.61
1,Andalucía (AN),2021,5.53
2,Aragón (AR),2021,8.45
3,"Asturias, Principado de (AS)",2021,7.73
4,"Balears, Illes (IB)",2021,5.82
